In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math
import seaborn as sns
from cvx.covariance.combination import from_ewmas
from arch import arch_model
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from scipy.optimize import minimize
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS

In [ ]:
#Ensemble des librairies utilisées
from xbbg import blp
import datetime
import numpy as np
import pandas as pd
import time
import os
import tensorflow as tf
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy import stats


from pylatex import Document, Section, Subsection, Math, Table, Tabular, Figure, Command, MiniPage
from pylatex.utils import NoEscape
import subprocess
from pylatex.utils import bold
from pylatex.package import Package
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from plotly.subplots import make_subplots
from tensorflow.keras.initializers import GlorotUniform
from keras.layers import Dropout
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from statsmodels.tsa.stattools import adfuller
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv(r'C:\Users\33640\OneDrive\Documents\GitHub\Covariance-Correlation-Volatility-Forecasting\SP500_daily_returns.csv')

In [3]:
nan_counts=df.isna().sum()
columns_to_keep = nan_counts[nan_counts <= 3].index
returns = df[columns_to_keep]
returns=returns.dropna()
returns.shape

(6224, 355)

In [4]:
dates=returns['Date']
returns=returns.drop('Date', axis=1)

In [8]:
def histo_cov(df, date, window):
    cov=np.zeros((len(df.columns),len(df.columns)))
    for i in range(date-window+1, date+1):
        cov+=np.outer(np.array(df.iloc[i,:]),np.array(df.iloc[i,:]))/(window)
    covs=pd.DataFrame(cov*250, columns=df.columns, index=df.columns)
    return(covs)

In [9]:
def ewma_cov(df, date, window, beta):
    cov=np.zeros((len(df.columns),len(df.columns)))
    for i in range(date-window+1, date+1):
        cov=(beta)*cov+(1-beta)*np.outer(np.array(df.iloc[i,:]),np.array(df.iloc[i,:]))
    covs=pd.DataFrame(cov*250, columns=df.columns, index=df.columns)
    return(covs)

In [58]:
halflife_pairs=[(10, 21), (21, 63), (63, 125)]
def ewma_cov(df, date, halflife_pairs, window):    
    combinator = from_ewmas(df.iloc[max(0,date-1000):date,:],
                                halflife_pairs,
                                min_periods_vola=window,  # min periods for volatility estimation
                                min_periods_cov=window)  # min periods for correlation estimation

    # Solve combination problem and loop through combination results to get predictors
    covariance_predictors = {}
    for predictor in combinator.solve(window=10):  # lookback window for optimization
        # From predictor we can access predictor.time, predictor.mean (=0 here),
        # predictor.covariance, and predictor.weights
        covariance_predictors[predictor.time] = predictor.covariance
    return(covariance_predictors[date]*250)
    

In [67]:
# Step 1: Fit GARCH(1,1) to each asset's returns
def fit_garch(returns):
    # Rescaler les rendements
    returns_rescaled = returns * 100
    
    model = arch_model(returns_rescaled, vol='Garch', p=1, q=1)
    garch_fit = model.fit(disp="off")
    
    # Revenir à l'échelle d'origine pour les résidus
    residuals = garch_fit.resid / 100
    volatilities = garch_fit.conditional_volatility / 100
    return volatilities, residuals
# Step 2: DCC-GARCH log-likelihood function
def dcc_garch_log_likelihood(params, residuals):
    """
    Calcule la log-vraisemblance du modèle DCC-GARCH.
    
    Paramètres:
    params (tuple): Paramètres alpha_2 et beta_2.
    residuals (np.array): Matrice des résidus standardisés (T x N).
    
    Retourne:
    float: La valeur de la log-vraisemblance négative.
    """
    alpha_2, beta_2 = params
    n_assets = residuals.shape[1]
    
    # Initialisation de Q_bar et Q_t
    T = residuals.shape[0]
    Q_bar = np.cov(residuals.T)  # Matrice de corrélation inconditionnelle
    Q_t = Q_bar.copy()
    
    log_likelihood = 0
    epsilon = 1e-5  # Petit terme de régularisation pour éviter les matrices singulières
    
    for t in range(T):
        # Mise à jour de Q_t selon la formule DCC
        Q_t = (1 - alpha_2 - beta_2) * Q_bar + alpha_2 * np.outer(residuals[t], residuals[t]) + beta_2 * Q_t
        
        # Régularisation de la diagonale de Q_t pour éviter les valeurs négatives
        diag_Q_t = np.diag(Q_t)
        diag_Q_t = np.clip(diag_Q_t, epsilon, None)  # Clip des valeurs négatives ou proches de zéro
        Q_t = np.diag(diag_Q_t) + (Q_t - np.diag(np.diag(Q_t)))  # Remplacer la diagonale
        
        # Ajouter un petit terme de régularisation à Q_t
        Q_t += epsilon * np.eye(n_assets)
        
        # Normaliser Q_t pour obtenir R_t (matrice de corrélation dynamique)
        D_t = np.diag(1 / np.sqrt(np.diag(Q_t)))
        R_t = D_t @ Q_t @ D_t
        
        # Vérifier les valeurs propres de R_t et corriger si nécessaire
        eigenvalues = np.linalg.eigvals(R_t)
        if np.any(eigenvalues <= 0):
            R_t += epsilon * np.eye(n_assets)
        
        # Calcul de la log-vraisemblance pour chaque étape t
        try:
            term1 = np.log(np.linalg.det(R_t))  # Log du déterminant de R_t
        except np.linalg.LinAlgError:
            # En cas de matrice singulière
            return np.inf  # Retourner une grande valeur pour pénaliser l'optimisation
        
        term2 = residuals[t].T @ np.linalg.pinv(R_t) @ residuals[t]  # epsilon_t' R_t^{-1} epsilon_t
        term3 = residuals[t].T @ residuals[t]  # epsilon_t' epsilon_t
        log_likelihood += term1 + term2 + term3  # Ajouter les trois termes pour chaque t
    
    # Retourner la log-vraisemblance négative
    return 0.5 * log_likelihood

# Step 6: Optimization of alpha_2 and beta_2
def optimize_dcc_garch(returns):
    """
    Optimise les paramètres alpha_2 et beta_2 du modèle DCC-GARCH en utilisant une fenêtre de données.
    
    Paramètres:
    returns (pd.DataFrame): DataFrame des rendements journaliers des actifs (chaque colonne représente un actif).
    window (str): Fenêtre temporelle pour la sélection des données ('6M' pour 6 mois, '1Y' pour 1 an, etc.).
    
    Retourne:
    tuple: Paramètres optimisés alpha_2 et beta_2.
    """
    n_assets = returns.shape[1]
    
    # Ajuster GARCH(1,1) sur chaque actif et standardiser les résidus
    volatilities = np.zeros_like(returns)
    residuals = np.zeros_like(returns)
    
    for i in range(n_assets):
        volatilities[:, i], residuals[:, i] = fit_garch(returns.iloc[:, i])
    
    standardized_residuals = residuals / volatilities
    
    # Valeurs initiales pour alpha_2 et beta_2
    initial_params = np.array([0.05, 0.9])
    
    # Contraintes sur les paramètres
    bounds = [(0, 1), (0, 0.95)]  # Limiter beta_2 à 0.95 pour éviter une convergence lente
    
    # Minimiser la log-vraisemblance négative
    result = minimize(dcc_garch_log_likelihood, initial_params, args=(standardized_residuals), bounds=bounds, method='SLSQP', options={'disp': True})
    
    if result.success:
        optimized_params = result.x
        return optimized_params
    else:
        raise ValueError("Optimization failed")

def predict_covariance_dcc_garch(df, date, window, horizon=1):
    """
    Calcule la matrice de covariance prédite par le modèle DCC-GARCH à un horizon donné.
    
    Paramètres:
    returns (pd.DataFrame): DataFrame des rendements journaliers des actifs (chaque colonne représente un actif).
    window (str): Fenêtre temporelle pour la sélection des données ('6M' pour 6 mois, '1Y' pour 1 an, etc.).
    horizon (int): Nombre de jours dans le futur pour lesquels la covariance doit être prédite.
    
    Retourne:
    pd.DataFrame: Matrice de covariance prédite à l'horizon spécifié.
    """
    returns=df.iloc[date-window:date,:]
    n_assets = returns.shape[1]
    
    # Ajuster GARCH(1,1) sur chaque actif et standardiser les résidus
    volatilities = np.zeros_like(returns)
    residuals = np.zeros_like(returns)
    
    for i in range(n_assets):
        volatilities[:, i], residuals[:, i] = fit_garch(returns.iloc[:, i])
    
    standardized_residuals = residuals / volatilities
    
    # Initialiser les paramètres DCC-GARCH avec les valeurs optimisées
    optimized_alpha2, optimized_beta2 = optimize_dcc_garch(returns)
    
    # Initialisation de Q_bar et Q_t
    Q_bar = np.cov(standardized_residuals.T)
    Q_t = Q_bar.copy()
    T = standardized_residuals.shape[0]
    
    # Utiliser les paramètres optimisés pour prédire la covariance
    for t in range(T):
        Q_t = (1 - optimized_alpha2 - optimized_beta2) * Q_bar + optimized_alpha2 * np.outer(standardized_residuals[t], standardized_residuals[t]) + optimized_beta2 * Q_t
    
    # Normaliser Q_t pour obtenir la matrice de corrélation dynamique R_t à la dernière date
    D_t = np.diag(1 / np.sqrt(np.diag(Q_t)))
    R_t = D_t @ Q_t @ D_t
    
    # Prédire la matrice de covariance à horizon k jours
    last_volatilities = volatilities[-1, :]
    D_last = np.diag(last_volatilities)
    predicted_covariance = D_last @ R_t @ D_last
    
    # Boucle pour étendre la prédiction à horizon k jours
    for _ in range(horizon):
        # Mettre à jour Q_t pour chaque jour à venir
        Q_t = (1 - optimized_alpha2 - optimized_beta2) * Q_bar + optimized_beta2 * Q_t
        
        # Normaliser Q_t pour obtenir R_t (corrélation dynamique après k jours)
        D_t = np.diag(1 / np.sqrt(np.diag(Q_t)))
        R_t = D_t @ Q_t @ D_t
        
        # Mise à jour des volatilités pour chaque jour
        predicted_covariance = D_last @ R_t @ D_last
    
    # Retourner la matrice de covariance sous forme de DataFrame avec les colonnes et index des actifs
    return pd.DataFrame(predicted_covariance*250, index=returns.columns, columns=returns.columns)

In [18]:
def reg_RW_vols(df, date, windows):
    Har_vols = []
    for actif in df.columns:
        vols=[[] for i in range(len(windows))]
        vol_realized = []
        M=max(windows)
        for t in range(date-250, date - M):
            for i, window in enumerate(windows):
                if window>1:
                    vols[i].append(df.loc[t - window+1:t, actif].std() * np.sqrt(250))
                else:
                    vols[i].append(np.sqrt(df.loc[t , actif]**2 * 250))
            vol_realized.append(np.sqrt(df.loc[t+1 , actif]**2 * 250))
        vol_data = pd.DataFrame({f'vol_{i+1}': vols[i] for i in range(len(windows))})
        vol_data['vol_realized'] = vol_realized
        vol_data = vol_data.dropna()
        
        X = vol_data[[f'vol_{i+1}' for i in range(len(windows))]]
        X = add_constant(X)
        y = vol_data['vol_realized']
        model = OLS(y, X).fit()
        latest_vols={}
        latest_vols['intercept'] = [1]
        for i, window in enumerate(windows):
            if window>1:
                latest_vols[f'vol_{i+1}']=[df.loc[date - window+1:date, actif].std() * np.sqrt(250)] 
            else:
                latest_vols[f'vol_{i+1}']=[np.sqrt(df.loc[date, actif]**2 * np.sqrt(250))] 
        
        latest_data = pd.DataFrame(latest_vols)
        vol_forecast = model.predict(latest_data)
        Har_vols.append(vol_forecast.iloc[0])
    
    return (np.array(Har_vols))

In [20]:
def reg_model_vols(df, date, windows):
    def ewma_vol(df, date, actif, beta):
        vol=0
        for t in range(date-100+1,date+1):
            vol=beta*vol+(1-beta)*df.loc[t, actif]**2
        return(np.sqrt(vol*250))
    def ewma_ret(df, date, actif, beta):
        ret=0
        for t in range(date-50+1,date+1):
            ret=beta*ret+(1-beta)*df.loc[t, actif]
        return(ret*250)
    Har_vols = []
    for actif in df.columns:
        vols = [[] for i in range(len(windows))]
        vol_realized = []
        ewma_vols = []
        ewma_returns = []
        neg_vols = []
        avg_vols = []
        M = max(windows)
        
        # Boucle sur la période pour construire les variables
        for t in range(date - 250, date - M):
            for i, window in enumerate(windows):
                if window>1:
                    vols[i].append(df.loc[t - window+1:t, actif].std() * np.sqrt(250))
                else:
                    vols[i].append(np.sqrt(df.loc[t , actif]**2 * 250))
            vol_realized.append(np.sqrt(df.loc[t+1 , actif]**2 * 250))

            ewma_vols.append(ewma_vol(df, t, actif, 0.94))
            ewma_returns.append(ewma_ret(df, t, actif, 0.94))

            # Calcul de la volatilité des rendements négatifs sur 5 jours
            neg_returns = df.loc[t - 4:t, actif][df.loc[t - 4:t, actif] < 0]
            if len(neg_returns) == 0:
                neg_vols.append(0)
            elif len(neg_returns) == 1:
                neg_vols.append(np.sqrt(neg_returns.iloc[0] ** 2 * 250))
            else:
                neg_vols.append(neg_returns.std() * np.sqrt(250))

            # Calcul de la volatilité moyenne sur 5 jours pour tous les actifs
            avg_vols.append(df.loc[t - 4:t].std(axis=1).mean() * np.sqrt(250))

        # Mettre les données en DataFrame
        vol_data = pd.DataFrame({f'vol_{i+1}': vols[i] for i in range(len(windows))})
        vol_data['vol_realized'] = vol_realized
        vol_data['ewma_vol'] = ewma_vols
        vol_data['ewma_return'] = ewma_returns
        vol_data['neg_vol'] = neg_vols
        vol_data['avg_vol'] = avg_vols
        vol_data = vol_data.dropna()

        # Régression linéaire
        X = vol_data[[f'vol_{i+1}' for i in range(len(windows))] + ['ewma_vol', 'ewma_return', 'neg_vol', 'avg_vol']]
        X = add_constant(X)
        y = vol_data['vol_realized']
        model = OLS(y, X).fit()

        # Prédiction des volatilités futures
        latest_vols = {'intercept': [1]}
        for i, window in enumerate(windows):
            if window > 1:
                latest_vols[f'vol_{i+1}'] = [df.loc[date - window + 1:date, actif].std() * np.sqrt(250)]
            else:
                latest_vols[f'vol_{i+1}'] = [np.sqrt(df.loc[date, actif] ** 2 * 250)]
        
        latest_vols['ewma_vol'] = [ewma_vol(df, date, actif, 0.94)]
        latest_vols['ewma_return'] = [ewma_ret(df, date, actif, 0.94)]
        
        neg_returns = df.loc[date - 4:date, actif][df.loc[date - 4:date, actif] < 0]
        if len(neg_returns) == 0:
            latest_vols['neg_vol'] = [0]
        elif len(neg_returns) == 1:
            latest_vols['neg_vol'] = [np.sqrt(neg_returns.iloc[0] ** 2)* 250]
        else:
            latest_vols['neg_vol'] = [neg_returns.std() * np.sqrt(250)]
        
        latest_vols['avg_vol'] = [df.loc[date - 4:date].std(axis=1).mean() * np.sqrt(250)]
        
        lastest_data = pd.DataFrame(latest_vols)
        vol_forecast = model.predict(lastest_data)
        Har_vols.append(vol_forecast.iloc[0])
    
    return np.array(Har_vols)

In [21]:
reg_RW_vols(returns.iloc[:,:5], 1000, [1,5,21])

array([0.27234684, 0.25998363, 0.14852436, 0.13697378, 0.29743358])

In [22]:
reg_model_vols(returns.iloc[:,:5],  1000, [1,5,21])

array([0.30784448, 0.33990836, 0.1609533 , 0.14720231, 0.28712086])

In [17]:
np.sqrt(returns.iloc[1000:1005,:5].cov()*250)

c:\Users\33640\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in sqrt
  result = func(self.values, **kwargs)


,A,AAPL,ABT,ACGL,ADBE
A,0.269033,0.239699,0.074348,NaN,0.143816
AAPL,0.239699,0.282170,0.058820,NaN,0.215717
ABT,0.074348,0.058820,0.113243,NaN,0.108515
ACGL,NaN,NaN,NaN,0.141759,NaN
ADBE,0.143816,0.215717,0.108515,NaN,0.209910


In [7]:
class Cov_Models:
    def __init__(self, df, annualization):
        self.df = df
        self.annualization = annualization 
    def histo_cov(self, window):
        if window == 1:
            # Approxime la covariance en utilisant le produit des rendements pour chaque paire d'actifs
            cov_matrix = pd.DataFrame(index=self.df.columns, columns=self.df.columns)
            for col1 in self.df.columns:
                for col2 in self.df.columns:
                    cov_matrix.loc[col1, col2] = self.df.iloc[-1][col1] * self.df.iloc[-1][col2] * self.annualization
        else:
            # Calcul standard de la covariance sur la fenêtre définie
            cov_matrix = self.df.iloc[len(self.df.index) - window:len(self.df.index), :].cov() * self.annualization

        return cov_matrix
    
    def EWMA_series(self,beta, actif1,actif2):
        ewma_cov=0
        m1 = max(self.df.loc[:, actif1].first_valid_index(), self.df.loc[:, actif2].first_valid_index())
        while self.df.loc[m1:, actif1].isna().any() or self.df.loc[m1:, actif2].isna().any():
            m1+=1
        for t in range(m1,len(self.df.index)):
            r_t1 = self.df.loc[t,actif1]
            r_t2 = self.df.loc[t,actif2]
            ewma_cov=beta*ewma_cov+(1-beta)* r_t1*r_t2
        ewma_cov=ewma_cov*self.annualization
        return (ewma_cov)
    def EWMA_cov_opti(self, actif1,actif2, horizon):
        betas = np.linspace(0.90, 0.99, 21)
        covs = [[] for _ in range(len(betas))]
        score= [[] for _ in range(len(betas))]
        for j, beta in enumerate(betas):
            cov_realized = []
            for t in range(len(self.df.index)-100-horizon,len(self.df.index)-horizon, 10):
                realized=0
                for j in range(horizon):
                    realized+= self.df.loc[t+j:t + j, actif1]* self.df.loc[t+j:t + j, actif2] * self.annualization
                cov_realized.append(realized)
                covs[j].append(self.EWMA_series(beta, actif1,actif2))
            score[j]=np.sum((np.array(cov_realized)-np.array(covs[j]))**2)
        beta_opt=betas[score.index(min(score))]
        return(beta_opt)
    def EWMA_cov(self, beta_var, beta_cov, horizon):
        ewma_cov=[]
        pairs=[]
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                pairs.append((self.df.columns[j], self.df.columns[i]))
        for j, (actif1, actif2) in enumerate (pairs):
            if beta_var=='Opti' and beta_cov=='Opti':
                ewma_cov.append(self.EWMA_series(self.EWMA_cov_opti( actif1,actif2, horizon), actif1,actif2))
            else:
                if actif1==actif2:
                    ewma_cov.append(self.EWMA_series(beta_var, actif1,actif2))
                else:
                    ewma_cov.append(self.EWMA_series(beta_cov, actif1,actif2))
        covs=np.zeros((len(self.df.columns),len(self.df.columns)))
        k=0
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                covs[j,i]=covs[i,j]=ewma_cov[k]
                k+=1
        covariance=pd.DataFrame(covs, columns=self.df.columns, index=self.df.columns)
        return covariance
    
    def HAR_cov(self, windows_var, windows_cov, horizon):
        Har_covs = []
        pairs = []
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                pairs.append((self.df.columns[j], self.df.columns[i]))
        for actif1, actif2 in pairs:
            m1 = max(self.df.loc[:, actif1].first_valid_index(), self.df.loc[:, actif2].first_valid_index())
            while self.df.loc[m1:, actif1].isna().any() or self.df.loc[m1:, actif2].isna().any():
                m1 += 1
            if actif1 == actif2:
                windows = windows_var
            else:
                windows = windows_cov
            covs = [[] for _ in range(len(windows))]
            cov_realized = []
            for t in range(max(max(windows[-1], m1),len(self.df.index) - 250),  len(self.df.index) - horizon):
                for i, window in enumerate(windows):
                    if window > 1:
                        cov_value = self.df.loc[t - window:t - 1, [actif1, actif2]].cov().iloc[0, 1] * self.annualization
                    else:
                        # Calcul de covariance pour une fenêtre de taille 1
                        cov_value = self.df.loc[t - 1, actif1] * self.df.loc[t - 1, actif2] * self.annualization
                    covs[i].append(cov_value)

                # Calcul pour cov_realized
                if horizon > 1:
                    cov_realized_value = self.df.loc[t:t + horizon, [actif1, actif2]].cov().iloc[0, 1] * self.annualization
                else:
                    cov_realized_value = self.df.loc[t, actif1] * self.df.loc[t, actif2] * self.annualization
                cov_realized.append(cov_realized_value)

            cov_data = pd.DataFrame({f'cov_{i+1}': covs[i] for i in range(len(windows))})
            cov_data['cov_realized'] = cov_realized
            cov_data = cov_data.dropna()

            X = cov_data[[f'cov_{i+1}' for i in range(len(windows))]]
            X = add_constant(X)
            y = cov_data['cov_realized']
            model = OLS(y, X).fit()
            latest_covs = {'intercept': [1]}
            for i, window in enumerate(windows):
                if window > 1:
                    latest_cov_value = self.df.loc[len(self.df.index) - window:len(self.df.index) - 1, [actif1, actif2]].cov().iloc[0, 1] * self.annualization
                else:
                    latest_cov_value = self.df.loc[len(self.df.index) - 1, actif1] * self.df.loc[len(self.df.index) - 1, actif2] * self.annualization
                latest_covs[f'cov_{i+1}'] = [latest_cov_value]

            lastest_data = pd.DataFrame(latest_covs)
            cov_forecast = model.predict(lastest_data)
            Har_covs.append(cov_forecast.iloc[0])
            if progress_bar:  # Update the progress bar if provided
                progress_bar.update(1)
        covs = np.zeros((len(self.df.columns), len(self.df.columns)))
        k = 0
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                covs[j, i] = covs[i, j] = Har_covs[k]
                k += 1
        covariance = pd.DataFrame(covs, columns=self.df.columns, index=self.df.columns)
        return covariance
        
    def timeseries_to_supervised(self, data, lag=1):
        supervised_data = pd.DataFrame(data)
        columns = [supervised_data.shift(i) for i in range(1, lag+1)]
        columns.append(supervised_data)
        supervised_data = pd.concat(columns, axis=1)
        supervised_data.fillna(0, inplace=True)
        supervised_data = supervised_data[2:].reset_index(drop=True)
        return supervised_data

    def fit_lstm(self, X, y, batch_size, nb_epoch, neurons):
        # Define the model with the preferred Input(shape) object
        model = Sequential()
        model.add(Input(shape=(X.shape[1], X.shape[2])))  # Using Input layer to define input shape
        model.add(LSTM(neurons))
        model.add(Dropout(0.2))
        model.add(Dense(1))

        # Compile the model
        model.compile(loss='mean_squared_error', optimizer='adam')

        # Fit the model
        for _ in range(nb_epoch):
            model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)

        return model

    def LSTM_cov(self,  batch_size, nb_epoch, neurons, horizon):
        cov = []
        pairs=[]
        Stationnary={}
        for actif in self.df.columns:
            rendements=self.df.loc[:,actif]
            m1=rendements.first_valid_index()
            while rendements.iloc[m1:].isna().any():
                m1+=1
            y=rendements.iloc[m1:]
            adf_result = adfuller(y)
            if adf_result[1]>=0.05:
                Stationnary[actif]=False
            else:
                Stationnary[actif]=True
        k=0
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                k+=1
                pairs.append((self.df.columns[j], self.df.columns[i]))
        k=0
        for actif1, actif2 in pairs:
            if Stationnary[actif1]==False or Stationnary[actif2]==False:
                cov.append(self.df.loc[len(self.df.index)-horizon*self.annualization:len(self.df.index), [actif1,actif2]].cov().iloc[0,1] * self.annualization)
            else:
                m1 = max(self.df.loc[:, actif1].first_valid_index(), self.df.loc[:, actif2].first_valid_index())
                while self.df.loc[m1:, actif1].isna().any() or self.df.loc[m1:, actif2].isna().any():
                    m1+=1
                constante1 = self.df.loc[0:len(self.df.index)-96, [actif1,actif2]].cov().iloc[0,1] * self.annualization
                constante2 = self.df.loc[len(self.df.index)-96:len(self.df.index), [actif1,actif2]].cov().iloc[0,1] * self.annualization
                high = max(constante1, constante2)
                low = min(constante1, constante2)

                if horizon == 1:
                    if (high - low) / low >= 0.3:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(len(self.df.index)-108, m1), len(self.df.index), 6)])
                    else:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(m1, 5), len(self.df.index), 6)])
                        ind_to_remove = np.argsort(X)[-4:]
                        mask = np.ones(len(X), dtype=bool)
                        mask[ind_to_remove] = False
                        X = X[mask]
                elif horizon == 3:
                    if (high - low) / low >= 0.3:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(len(self.df.index)-144, m1), len(self.df.index), 6)])
                    else:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(m1, 5), len(self.df.index), 6)])
                        ind_to_remove = np.argsort(X)[-3:]
                        mask = np.ones(len(X), dtype=bool)
                        mask[ind_to_remove] = False
                        X = X[mask]
                elif horizon == 5 or horizon == 7:
                    if (high - low) / low >= 0.3:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(len(self.df.index)-180, m1), len(self.df.index), 6)])
                    else:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(m1, 5), len(self.df.index), 6)])
                        ind_to_remove = np.argsort(X)[-2:]
                        mask = np.ones(len(X), dtype=bool)
                        mask[ind_to_remove] = False
                        X = X[mask]
                elif horizon == 10:
                    if (high - low) / low >= 0.3:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(len(self.df.index)-240, m1), len(self.df.index), 6)])
                    else:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(m1, 5), len(self.df.index), 6)])
                        ind_to_remove = np.argsort(X)[-1:]
                        mask = np.ones(len(X), dtype=bool)
                        mask[ind_to_remove] = False
                        X = X[mask]

                X = X.reshape(len(X), 1)
                scaler = MinMaxScaler(feature_range=(-1, 1))
                scaler = scaler.fit(X)
                scaled_X = scaler.transform(X)
                train = self.timeseries_to_supervised(scaled_X, lag=1)
                values = train.values
                X, y = values[:, 0].reshape(-1, 1), values[:, 1].reshape(-1, 1)
                X = X.reshape((X.shape[0], 1, X.shape[1]))

                model = self.fit_lstm(X, y, batch_size, nb_epoch, neurons)
                cov_forecast = model.predict(X, batch_size=1)
                cov.append(scaler.inverse_transform(cov_forecast)[:2*horizon, 0].mean())
                k+=1
                if progress_bar:  # Update the progress bar if provided
                    progress_bar.update(1)
        covs=np.zeros((len(self.df.columns),len(self.df.columns)))
        k=0
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                covs[j,i]=covs[i,j]=cov[k]
                k+=1
                
        covariance=pd.DataFrame(covs, columns=self.df.columns, index=self.df.columns)
        volatilities = np.sqrt(np.diag(covariance))

        # Parcourir chaque élément de la matrice
        for i in range(covariance.shape[0]):
            for j in range(covariance.shape[1]):
                # Calculer le produit des volatilités correspondantes
                vol_product = volatilities[i] * volatilities[j]
                # Vérifier si la covariance est supérieure au produit des volatilités
                if covariance.iloc[i, j] > vol_product:
                    # Remplacer la valeur par le produit des volatilités
                    covariance.iloc[i, j] = vol_product
        return covariance

    def volatilite(self, covariance):
        vol=[]
        for i in range(covariance.shape[0]):
            vol.append(np.sqrt(covariance.iloc[i,i]))
        return(np.array(vol))
    
    def correlation(self, covariance):
        correl= np.zeros((covariance.shape[0],covariance.shape[0]))
        vol=self.volatilite(covariance)
        for i in range(covariance.shape[0]):
            for j in range(covariance.shape[1]):
                correl[i,j]=covariance.iloc[i,j]/(vol[i]*vol[j])
        return(pd.DataFrame(correl, columns=self.df.columns, index=self.df.columns))